# Keras text 전처리

keras는 기본적인 전처리를 위한 도구를 제공한다\
integer encoding을 위해 keras 전처리 도구인 tokenizer를 사용한다

In [5]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
raw_text = " A barber is a person. a barber is good person. a barber is huge person.\
            he knew A Secret! The Secret He Kept is huge secret. Huge Secret. His barber kept his word.\
            a barber kept his word.His barber kept his secret.\
            But keeping and keeping such a huge secret to himself was driving the barber crazy.\
            the barber went up a huge mountain."
sentences = sent_tokenize(raw_text)

vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english'))

for sentence in sentences:
    # 단어 토큰화 수행
    tokenized_sentence = word_tokenize(sentence)
    result = []
    
    for word in tokenized_sentence:
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄여준다.
        if word not in stop_words:  # 단어 token화 된 결과에 대해서 불용어 제거
            if len(word)>2: # 단어 길이가 2 이하라면 제거
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0
                vocab[word] += 1
    preprocessed_sentences.append(result)

In [9]:
preprocessed_sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word.his', 'barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

단어 token화까지 수행된 앞에서 사용한 text daata와 동일한 data를 사용

In [10]:
tokenizer = Tokenizer()

# fit_on_texts() 안에 corpus를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성
tokenizer.fit_on_texts(preprocessed_sentences)

fits_on_texts는 입력한 text로 부터 단어 빈도수ㅜ가 높은 순으로 낮은 정수 index를 부여 하는데 정수 encoding 작업이 이루어진다고 생각하면 됨\
각 단어에 index가 어떻게 부여되었는지를 보기 위해서는, word_index를 사용한다

In [ ]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'keeping': 6, 'good': 7, 'knew': 8, 'word': 9, 'word.his': 10, 'driving': 11, 'crazy': 12, 'went': 13, 'mountain': 14}


각 단어의 빈도수가 높은 순서대로 index가 부여됨.\
각 단어가 count를 수행하였을 때, 몇 개였는지 보고 싶다면 word_counts를 사용

In [12]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 1), ('word.his', 1), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


texts_to_sequences()는 입력으로 들어온 corpus에 대해서 각 단어를 이미 정해진 index로 변환

In [13]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 7, 5], [1, 3, 5], [8, 2], [2, 4, 3, 2], [3, 2], [1, 4, 9], [1, 4, 10, 1, 4, 2], [6, 6, 3, 2, 11, 1, 12], [1, 13, 3, 14]]


빈도수가 가장 높은 단어 n개만을 사용하기 위해 most_common()을 사용했었는데\
keras기반의 tokenizer에서는 tokenizer = Tokenizer(num_words=숫자) 와 같은 방식으로 빈도수가 높은 상위 몇 개의 단어만 사용하겠다고 지정이 가능\
1번 단어부터 5번 단어까지 사용해볼 것.\
그러기 위해서는 tokenizer를 재정의 해야함

In [14]:
vocab_size = 5
tokenizer = Tokenizer(num_words= vocab_size + 1)    # 상위 5갱의 단어만 사용할 것
tokenizer.fit_on_texts(preprocessed_sentences)

num_words에서 1을 더해주는 이유: num_words는 0부터 숫자를 count하기 때문\
그래서 5를 넣으면 0부터 4번 단어까지만 보존하므로 1번 단어부터 4번 단어까지만 남게 된다.\
1~5번 단어를 사용하고 싶다 => num_words에 5를 넣는게 아니라 5+1의 값을 넣어주어야 한다.

숫자 0에 지정된 단어가 존재하지 않는데 keras tokenizer가 숫자 0까지 단어 집합의 크기로 산정하는 이유 = padding

**따라서 keras tokenizer를 사용한다면 숫자 0도 단어 집합의 크기로 고려해야 한다**

In [15]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'keeping': 6, 'good': 7, 'knew': 8, 'word': 9, 'word.his': 10, 'driving': 11, 'crazy': 12, 'went': 13, 'mountain': 14}


상위 5개의 단어만 사용하겠다 했는데 13개의 단어가 모두 출력됨\
word_counts를 확인해보자

In [16]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 1), ('word.his', 1), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


word_counts에서도 13개의 단어가 모두 출력되는 것을 확인할 수 있었다.\
상위 n개의 단어만 보존되는 것의 실제 적용은  texts_to_seqeuences를 사용할 때 적용이 된다. 

In [17]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4, 1, 4, 2], [3, 2, 1], [1, 3]]


corpus에 대해 각 단어를 이미 정해진 index로 변환하는데, 상위 5개의 단어만을 사용하겠다고 지정하였으며 1번 단어부터 5번 단어까지만 보존되고 나머지 단어들은 제거\
word_index와 word_counts에서도 지정된 num_words만큼의 단어만 남기고 싶다면 다음의 방법을 사용하자

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)

In [20]:
vocab_size = 5
words_frequency = [word for word, index in tokenizer.word_index.items() if index >= vocab_size + 1]

# index가 5 초과인 단어 제거(index가 작을 수록 빈번하게 등장한 단어)
for word in words_frequency:
    del tokenizer.word_index[word]  # 해당 단어에 대한 index 정보를 삭제
    del tokenizer.word_counts[word] # 해당 단어에 대한 count 정보를 삭제
    
print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(preprocessed_sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4, 1, 4, 2], [3, 2, 1], [1, 3]]


keras tokenizer는 단어 집합에 없는 단어인 OOV에 대해서는 단어를 정수로 바꾸는 과정에서 아예 단어를 제거\
단어 집합에 없는 단어들은 OOV로 간주하여 보존하고 싶다면 Tokenizer의 인자 **oov_token**을 사용

In [21]:
# 숫자 0과 OOV를 고려하여 단어 집합의 크기를 +2 해주자

vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

oov_token을 사용한다면 keras tokenizer는 'OOV'의 index를 1로

In [22]:
print('단어 OOv의 index : {}', format(tokenizer.word_index['OOV']))

단어 OOv의 index : {} 1


corpups에 대해 integer encoding

In [23]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1, 2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


빈도수 상위 5개의 단어는 2~6까지의 index를 가지고, 그 외 단어 집합에 없는 'good'과 같은 단어들은 OOV의 index인 1로 encoding이 되었다.